In [17]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.models import resnet50
import numpy as np
import pandas as pd
import cv2
import os
import pickle
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from torchsummary import summary
from tensorboard.plugins import projector
from scipy.optimize import linear_sum_assignment
from utils.utils import *
from utils.assignment import *
from utils.latent_loss import *
from model.model import *

In [43]:
class SetPredictLSP(nn.Module):
    def __init__(self, device, hidden_dim=384, nhead=4, nlayers=3, max_sentence=18, vocab_size=5439):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.cnn_text = CNN_Text(device=device)
        self.hungarian = MSEGCRLatentLoss()
        
        self.decoder = nn.TransformerDecoderLayer(d_model=hidden_dim, nhead=nhead)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder, nlayers)
        
        self.linear = nn.Linear(hidden_dim, 128)
        self.output = nn.Linear(128, vocab_size)
        
        self.mlp = MLP().to(device)
        self.mlp.load_state_dict(torch.load('model/length_model_best.pt'))
        
        self.max_t = max_sentence
        
    def forward(self, X, y, len_y):
        '''
        X is image (bs, 3, H, W)
        R's shape is (bs, length, hidden_dim)
        feat will be use for the transformer decoder
        '''
        bs = X.shape[0]
        R, feat = self.cnn_text(X) 
        
        R = flat_by_lengths(R, (torch.ones((bs))*20).to(device))
        y = flat_by_lengths_max_t(y, len_y, self.max_t)
        R_pi, R_i, loss = self.hungarian.forward(y, len_y, R, (torch.ones(bs, dtype=int)*20).to(device))
        
        R_pi = chunk_pad_by_lengths(R_pi, len_y, batch_first = False)
        R = self.transformer_decoder(R_pi, feat).transpose(0, 1)
        R = flat_by_lengths(R, len_y)
        x = self.linear(R)
        output = self.output(x)
        
        return output, loss
    
    def predict(self, X):
        R, feat = self.cnn_text(X) 
        len_R = self.mlp(X)
        len_R = torch.round(len_R).flatten()
        
        R = self.transformer_decoder(R.transpose(0, 1), feat).transpose(0, 1)
        R = flat_by_lengths_max_t(R, len_R, 20)
        x = self.linear(R)
        output = self.output(x)
        
        return output, len_R

In [7]:
model = SetPredictLSP(device).to(device)

In [8]:
for X, y, id_, len_y in train_loader:
    break

In [9]:
print(X.shape)

torch.Size([81, 3, 256, 256])


In [10]:
output, len_R = model.predict(X.to(device))

In [16]:
R_pi = chunk_pad_by_lengths(output, torch.Tensor.int(len_R.flatten()), batch_first = True)

In [17]:
R_pi.shape

torch.Size([81, 7, 5439])

In [28]:
test = torch.argmax(R_pi[0][:int(len_R[0])], axis=1).

In [30]:
test = test.cpu().detach().numpy()

In [31]:
decode = lambda x: decoder[x]

list(map(decode, test))

['there is spondylosis',
 'mediastinum normal',
 'negative for pneumothorax or pleural effusion']

In [39]:
id__ = torch.argmax(id_[0][:int(len_y[0])], axis=1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [42]:
list(map(decode, id_[0][:int(len_y[0])].cpu().detach().numpy()))

['lungs are clear',
 'no focal consolidation effusion or pneumothorax',
 'interval resolution of left effusion',
 'central venous dialysis catheter unchanged in position',
 'heart and mediastinal contours are normal',
 'osseous structures intact']

In [36]:
len_y[0]

tensor(6, dtype=torch.int32)

In [ ]:
len_R.sum()

tensor(311., device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
candidates = [['this', 'is', 'a', 'test']]
score = corpus_bleu(references, candidates)
print(score)

In [ ]:
test = torch.argmax(R_pi[0][:int(len_R[0])], axis=1).

In [ ]:
decode = lambda x: decoder[x]

list(map(decode, test))

In [5]:
a = ['there is spondylosis',
 'mediastinum normal',
 'negative for pneumothorax or pleural effusion']
     
b = ['lungs are clear',
 'no focal consolidation effusion or pneumothorax',
 'interval resolution of left effusion',
 'central venous dialysis catheter unchanged in position',
 'heart and mediastinal contours are normal',
 'osseous structures intact']

In [6]:
split = lambda x: x.split()

a = list(map(split, a))
b = list(map(split, b))

In [11]:
a

[['there', 'is', 'spondylosis'],
 ['mediastinum', 'normal'],
 ['negative', 'for', 'pneumothorax', 'or', 'pleural', 'effusion']]

In [8]:
b

[['lungs', 'are', 'clear'],
 ['no', 'focal', 'consolidation', 'effusion', 'or', 'pneumothorax'],
 ['interval', 'resolution', 'of', 'left', 'effusion'],
 ['central', 'venous', 'dialysis', 'catheter', 'unchanged', 'in', 'position'],
 ['heart', 'and', 'mediastinal', 'contours', 'are', 'normal'],
 ['osseous', 'structures', 'intact']]

In [12]:
a = [a]

In [13]:
a

[[['there', 'is', 'spondylosis'],
  ['mediastinum', 'normal'],
  ['negative', 'for', 'pneumothorax', 'or', 'pleural', 'effusion']]]

In [14]:
score = corpus_bleu(a, b)

AssertionError: The number of hypotheses and their reference(s) should be the same 

In [82]:
a = [['contrast within the bilateral renal collecting systems']]
b = ['contrast is seen within the bilateral kidneys from prior examination']

In [83]:
a = [a[0][0].split()]
b = b[0].split()

In [86]:
score = sentence_bleu([b], a[0])

In [87]:
score

3.515018170403913e-78

In [20]:
b

[['lungs', 'are', 'clear'],
 ['no', 'focal', 'consolidation', 'effusion', 'or', 'pneumothorax'],
 ['interval', 'resolution', 'of', 'left', 'effusion'],
 ['central', 'venous', 'dialysis', 'catheter', 'unchanged', 'in', 'position'],
 ['heart', 'and', 'mediastinal', 'contours', 'are', 'normal'],
 ['osseous', 'structures', 'intact']]

In [32]:
a[0]

[['there', 'is', 'spondylosis'],
 ['mediastinum', 'normal'],
 ['negative', 'for', 'pneumothorax', 'or', 'pleural', 'effusion']]

In [33]:
b[0]

['lungs', 'are', 'clear']